In [2]:
from lxml import etree
from collections import OrderedDict
import pickle
import numpy as np
import re

In [5]:
datamap=OrderedDict()
# datamap["AE"]="C:/Users/harshitgujral/Desktop/Stack/data/android.stackexchange.com"
# datamap["DB"]="C:/Users/harshitgujral/Desktop/Stack/data/dba.stackexchange.com"
# datamap["SE"]="C:/Users/harshitgujral/Desktop/Stack/data/softwareengineering.stackexchange.com"
# datamap["SF"]= "C:/Users/harshitgujral/Desktop/Stack/data/serverfault.com"
# datamap["SU"]="C:/Users/harshitgujral/Desktop/Stack/data/superuser.com"
datamap["SO"]="C:/Users/harshitgujral/Desktop/Stack/data/stackoverflow.com-"

tagfile="tags/Logging-topic-modelling - Sheet3.csv"
mallet_location="C:/Mallet/Stack/"

logtags=list(set(read_tags(tagfile)))

In [4]:
def read_tags(filename):
    logtags=[]
    fn=open(filename,"r")
    next(fn)   # SKipping the first line
    for row in fn:
        logtags.append(row.split(",")[0].strip().lower())   
    return logtags

def atleast_one(a, b):
    return not set(a).isdisjoint(b)

def refine_tags(tags):
    if tags!=None:
        l=tags.split('><')
        l=[i.replace('>','').replace('<','').lower() for i in l]
        return l
    else:
        return ["???"]

def distribute_posts(website,logtags):
    if website=="SO":
        datamap[website]=datamap[website]+"Posts"   
    
    f=open(datamap[website]+"/Posts.xml","rb")
    next(f)
    next(f)
    count=0
    for l in f:
        try:
            row=etree.XML(l)      
            posttypeid=row.get("PostTypeId") 
            acceptedanswerID=row.get("AcceptedAnswerId")
            tags=refine_tags(row.get("Tags"))
            
            if posttypeid=="1":
                if atleast_one(tags,logtags):
                    body=row.get("Body")
                    title=row.get("Title")
                    body=title+" "+body
                    body=process_string(body)
                    fo=open(mallet_location+website+"/"+str(count)+".txt","w")
                    count+=1
                    fo.write(body)
                    fo.close()

        except:
            print(l)
    f.close()
    print(count)
    return

def process_string(s):
    s=clean_code(s)
    s=remove_HTML(s)
    s=remove_miscell(s)
    s=remove_special(s)
    return s

def clean_code(s):
    s=s.replace("&lt;code&gt;","<<")
    s=s.replace("&lt;/code&gt;",">>")
    s=s.replace("&ltcode&gt","<<")
    s=s.replace("&lt/code&gt",">>")
    s=re.sub('<[^>]+>', '', s)
    return s

def remove_HTML(s):
    s=s.replace("&lt;","<")
    s=s.replace("&gt;",">")
    s=s.replace("&lt","<")
    s=s.replace("&gt",">")
    p = re.compile(r'<.*?>')
    return p.sub('', s)

def remove_miscell(s):
    s=s.replace("&quot;","")
    s=s.replace("&#xA;","")
    return s

def remove_special(s):
    s=re.sub('[^A-Za-z0-9]+', ' ', s)
    return s.strip()

In [6]:
for fname in list(datamap.keys()):
    distribute_posts(fname,logtags)

MemoryError: 